<a href="https://colab.research.google.com/github/itdusty/blood_cells_classification/blob/main/data_preparation_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data preparation
This module turns filesystem from 'dataset/{video_name}/{frame_name}' to 'dataset/{class}/{video_name}/{frame_name}'.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
import os
import shutil
import pandas as pd
import numpy as np

In [13]:
working_dir = "/content/drive/MyDrive/Cells classification"

In [14]:
os.chdir(working_dir)
os.getcwd()

'/content/drive/MyDrive/Cells classification'

In [4]:
old_dataset = 'dataset'
new_dataset = 'prepared_dataset'

In [6]:
dataframe = pd.read_csv('DataFrame.csv')
dataframe

,Unnamed: 0,files,erythrocytes,lymphocytes,high_erythrocytes,high_lymphocytes,blur,noise
0,0,traffic1,9,9,0,1,0,0
1,1,traffic2,13,3,1,0,0,0
2,2,traffic3,17,11,1,1,0,0
3,3,traffic4,1,5,0,0,0,0
4,4,traffic5,14,8,1,1,0,0


In [ ]:
shutil.rmtree(f"{working_dir}/{new_dataset}")

In [15]:
if not os.path.isdir(new_dataset):
    os.mkdir(new_dataset)
    os.chdir(new_dataset)
    os.mkdir('high_rbc_high_wbc')
    os.mkdir('high_rbc_low_wbc')
    os.mkdir('low_rbc_high_wbc')
    os.mkdir('low_rbc_low_wbc')
    os.chdir("..")
os.getcwd()

'/content/drive/MyDrive/Cells classification'

In [16]:
for index, row in dataframe.iterrows():
  if row['high_erythrocytes'] == 1 and row['high_lymphocytes'] == 1:
    class_name = 'high_rbc_high_wbc'
  elif row['high_erythrocytes'] == 1 and row['high_lymphocytes'] == 0:
    class_name = 'high_rbc_low_wbc'
  elif row['high_erythrocytes'] == 0 and row['high_lymphocytes'] == 1:
    class_name = 'low_rbc_high_wbc'
  else:
    class_name = 'low_rbc_low_wbc'

  shutil.copytree(src=f"{working_dir}/{old_dataset}/{row['files']}", dst=f"{working_dir}/{new_dataset}/{class_name}/{row['files']}")
  print(row['files'])

traffic1
traffic2
traffic3
traffic4
traffic5
